In [8]:
# -*- coding: utf-8 -*-

import os
import sys
from pprint import pprint

import ccxt
import pandas as pd

PHEMEX_API_KEY = os.getenv('API_KEY_PHE_TEST')
PHEMEX_API_SECRET = os.getenv('API_SECRET_PHE_TEST')

phemex = ccxt.phemex(
    {
        "apiKey": PHEMEX_API_KEY,
        "secret": PHEMEX_API_SECRET,
        'options': {'defaultType': 'swap'},
        "verbose": False,  # switch it to False if you don't want the HTTP log
    }
)
phemex.set_sandbox_mode(True)

def get_price(exchange, curr: str) -> float:
    """ 
    Takes 'BTC' and returns the ticker price for 'BTC/USDT', if 'USDT' is passed in it
    returns 1.0. In this example pass the phemex exchange object in
    """
    if curr == 'USDT':
        return 1.0
    else:
        tick = exchange.fetchTicker(curr+'/USDT')
        mid_point = tick['bid']
        return mid_point
  

def get_phemex_balances():
    """
    Returns a dataframe with current positions and values 
    NOTE: we do not care about the locked/free for the balance calculation
    all tokens are presumed to be available.
    """
    # to see what we are working with
    # pprint(phemex.fetch_balance())
    # print()
    params = {"code": "USDT"}
    phemexBalance = phemex.fetch_balance(params)
    # print(phemexBalance['total'])
    # print()
    # print(phemexBalance['free'])
    
    balances = []
    for symbol, value in phemexBalance['total'].items():
        if value > 0.0:
            # get the bid price from the ticker price
            bid_price = get_price(phemex, symbol);
            # load a list of dictionary entries for easy dataframe creation
            datum = {}
            datum['asset'] = symbol
            datum['free'] = value
            datum['locked'] = 0.0
            datum['total'] = value
            datum['price'] = bid_price 
            datum['balance'] = round(bid_price * datum['total'],2)
            datum['platform'] = 'Phemex'
            balances.append(datum)
    
    # print(balances)
    df = pd.DataFrame(balances)
    return df
    
    
df = get_phemex_balances()
# print(f'Phemex Balance: {df.balance.sum():,.2f}')
df

""
